In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

from alibi_detect.cd import ClassifierDrift
from alibi_detect.datasets import fetch_cifar10c, corruption_types_cifar10c

2023-07-03 16:22:16.400216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 16:22:16.491280: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-03 16:22:16.989358: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/vishesh/anaconda3/lib/:/home/vishesh/anaconda3/lib/python3.9/site-packages/nvidi

In [2]:
train_data_dir = "/home/vishesh/Desktop/datasets/ships-data/X_true_train_240_320"
train_ground_truth_dir = "/home/vishesh/Desktop/datasets/ships-data/Y_true_train_240_320"

test_data_path = "/home/vishesh/Desktop/datasets/ships-data/X_true_val_240_320/1.npy"
test_ground_truth_path = "/home/vishesh/Desktop/datasets/ships-data/Y_true_val_240_320/1.npy"

perturbation = "contrast"
strength = 1
perturbed_dataset_path = f"/home/vishesh/Desktop/datasets/ships-data/perturbed_test/varying/{perturbation}/{strength}/1.npy"

In [3]:
X_train = None
y_train = None


for i in range(1,7):
    print(i)
    input_data_path = os.path.join(train_data_dir, f"{i}.npy")
    ground_truth_path = os.path.join(train_ground_truth_dir, f"{i}.npy")
    
    if i == 1:
        X_train = np.load(input_data_path)
        y_train = np.load(ground_truth_path)
        continue
        
        
    new_input_data = np.load(input_data_path)
    new_ground_truth = np.load(ground_truth_path)
    

    X_train = np.vstack((X_train, new_input_data))
    y_train = np.hstack((y_train, new_ground_truth))
    
y_train = y_train.astype(int)

1
2
3
4
5
6


In [4]:
X_test = np.load(perturbed_dataset_path)
y_test = np.load(test_ground_truth_path).astype(int)

# perturbed_dataset = np.load(perturbed_dataset_path)
# X_test[-200:] = perturbed_dataset[-200:]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6000, 240, 320, 3) (6000,) (1000, 240, 320, 3) (1000,)


In [5]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input

tf.random.set_seed(0)

model = tf.keras.Sequential(
  [
      Input(shape=(240, 320, 3)),
      Conv2D(8, 3, strides=1, padding='same', activation=tf.nn.relu),
      Conv2D(16, 3, strides=1, padding='same', activation=tf.nn.relu),
      Conv2D(32, 3, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(64, 5, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(128, 3, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(2, activation='softmax')
  ]
)

# cd = ClassifierDrift(X_train, model, p_val=.05, train_size=.75, epochs=10, batch_size=1)
cd = ClassifierDrift(X_train, model, p_val=.05, n_folds=5, epochs=1)

2023-07-03 16:22:30.376690: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
Both `n_folds` and `train_size` specified. By default `n_folds` is used.


In [ ]:
labels = ['No!', 'Yes!']

preds = cd.predict(X_test[:500])
print('No corruption')
print('Drift? {}'.format(labels[preds['data']['is_drift']]))
print(f'p-value: {preds["data"]["p_val"]:.3f}')